<a href="https://colab.research.google.com/github/IsadoraPrevitalle/ProcessamentoLinguagemNatural/blob/main/Sumariza%C3%A7%C3%A3o_Luhn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import nltk
import string
import heapq #Mede as extensões das strings

In [ ]:
nltk.download('punkt') #pontuações

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
texto = '''A educação é um dos pilares fundamentais para o desenvolvimento de uma sociedade.
Por meio dela, as pessoas adquirem conhecimento, habilidades e valores que são essenciais para sua participação ativa na comunidade e no mercado de trabalho.
No entanto, o acesso à educação nem sempre é equitativo.
Em muitas partes do mundo, há disparidades significativas no acesso à educação de qualidade, com grupos marginalizados enfrentando barreiras adicionais.
Para combater essa desigualdade, políticas públicas voltadas para democratização do acesso à educação são essenciais.
Isso inclui não apenas o acesso físico às escolas, mas também a qualidade do ensino oferecido.
investimentos em infraestrutura educacional, capacitação de professores e desenvolvimento de currículos inclusivos são medidas importantes nesse sentido.'''

In [ ]:
texto

'A educação é um dos pilares fundamentais para o desenvolvimento de uma sociedade.\nPor meio dela, as pessoas adquirem conhecimento, habilidades e valores que são essenciais para sua participação ativa na comunidade e no mercado de trabalho.\nNo entanto, o acesso à educação nem sempre é equitativo.\nEm muitas partes do mundo, há disparidades significativas no acesso à educação de qualidade, com grupos marginalizados enfrentando barreiras adicionais.\nPara combater essa desigualdade, políticas públicas voltadas para democratização do acesso à educação são essenciais.\nIsso inclui não apenas o acesso físico às escolas, mas também a qualidade do ensino oferecido.\ninvestimentos em infraestrutura educacional, capacitação de professores e desenvolvimento de currículos inclusivos são medidas importantes nesse sentido.'

In [ ]:
# Retiram espaços em branco
texto = re.sub(r'\s', ' ', texto)

In [ ]:
texto

'A educação é um dos pilares fundamentais para o desenvolvimento de uma sociedade. Por meio dela, as pessoas adquirem conhecimento, habilidades e valores que são essenciais para sua participação ativa na comunidade e no mercado de trabalho. No entanto, o acesso à educação nem sempre é equitativo. Em muitas partes do mundo, há disparidades significativas no acesso à educação de qualidade, com grupos marginalizados enfrentando barreiras adicionais. Para combater essa desigualdade, políticas públicas voltadas para democratização do acesso à educação são essenciais. Isso inclui não apenas o acesso físico às escolas, mas também a qualidade do ensino oferecido. investimentos em infraestrutura educacional, capacitação de professores e desenvolvimento de currículos inclusivos são medidas importantes nesse sentido.'

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [ ]:
def preprocessamento(texto):
  texto_formatado = texto.lower() #Deixa texto em minusculo
  tokens = []
  for token in nltk.word_tokenize(texto_formatado): #Função quebra as palavras de um texto, cada palavra fuca em uma linha
    tokens.append(token)

  #Verifica cada palavra e a mantem caso a mesma não for stopword ou uma pontuação
  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]

  #Concatena as palavras em forma de texto, com a adição de espaço em branco entre elas
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()]) #Verifica se é um numero (digito)

  return texto_formatado

In [ ]:
texto_formatado = preprocessamento(texto)
texto_formatado

'educação pilares fundamentais desenvolvimento sociedade meio pessoas adquirem conhecimento habilidades valores essenciais participação ativa comunidade mercado trabalho entanto acesso educação sempre equitativo muitas partes mundo disparidades significativas acesso educação qualidade grupos marginalizados enfrentando barreiras adicionais combater desigualdade políticas públicas voltadas democratização acesso educação essenciais inclui apenas acesso físico escolas qualidade ensino oferecido investimentos infraestrutura educacional capacitação professores desenvolvimento currículos inclusivos medidas importantes nesse sentido'

In [ ]:
def calcula_notas_sentenca(sentencas, palavras_importantes, distancia):
  notas = []
  indices_sentencas = 0
  for sentenca in [nltk.word_tokenize(sentenca.lower()) for sentenca in sentencas]:
    indice_palavra = []
    for palavra in palavras_importantes:
      try:
        indice_palavra.append(sentenca.index(palavra)) # index de cada palavra (posição)
      except ValueError:
        pass

    indice_palavra.sort()

    if len(indice_palavra) == 0:
      continue

    lista_grupos = []
    grupo = [indice_palavra[0]]
    i = 1
    while i < len(indice_palavra):
      if indice_palavra[i] - indice_palavra[i - 1] < distancia:
        grupo.append(indice_palavra[i])
      else:
        lista_grupos.append(grupo[:])
        grupo = [indice_palavra[i]]
      i += 1
    lista_grupos.append(grupo)
    nota_max_grupo = 0
    for g in lista_grupos:
      palavras_importantes_no_grupo = len(g)
      total_palavra_no_grupo = g[-1] - g[0] + 1
      nota = 1.0 * palavras_importantes_no_grupo**2 / total_palavra_no_grupo
      if nota > nota_max_grupo:
        nota_max_grupo = nota

      notas.append((nota_max_grupo, indices_sentencas))
      indices_sentencas += 1

    return notas

In [ ]:
def sumarizar(texto, top_n_palavras, distancia, qtd_sentencas):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]
  palavras = [palavra for sentenca in sentencas_formatadas for palavra in nltk.word_tokenize(sentenca)]
  frequencia = nltk.FreqDist(palavras)
  top_n_palavras = [palavra[0] for palavra in frequencia.most_common(top_n_palavras)]
  notas_sentencas = calcula_notas_sentenca(sentencas_formatadas, top_n_palavras, distancia)
  melhores_sentencas = heapq.nlargest(qtd_sentencas, notas_sentencas)
  melhores_sentencas = [sentencas_originais[i] for (nota, i) in melhores_sentencas]

  return sentencas_originais, melhores_sentencas, notas_sentencas

In [ ]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(texto, 5, 3, 3)

In [ ]:
sentencas_originais

['A educação é um dos pilares fundamentais para o desenvolvimento de uma sociedade.',
 'Por meio dela, as pessoas adquirem conhecimento, habilidades e valores que são essenciais para sua participação ativa na comunidade e no mercado de trabalho.',
 'No entanto, o acesso à educação nem sempre é equitativo.',
 'Em muitas partes do mundo, há disparidades significativas no acesso à educação de qualidade, com grupos marginalizados enfrentando barreiras adicionais.',
 'Para combater essa desigualdade, políticas públicas voltadas para democratização do acesso à educação são essenciais.',
 'Isso inclui não apenas o acesso físico às escolas, mas também a qualidade do ensino oferecido.',
 'investimentos em infraestrutura educacional, capacitação de professores e desenvolvimento de currículos inclusivos são medidas importantes nesse sentido.']

In [ ]:
melhores_sentencas

In [ ]:
notas_sentencas

In [ ]:
from ipywidgets import HTML

In [ ]:
from ipywidgets import HTML
def visualiza_resumo (titulo, lista_sentencas, melhores_sentencas):
  from IPython.core.display import display_html
  texto = ''

  display(HTML(f'<h1> Resumo de texto - {titulo}</h1>'))
  for i in lista_sentencas:
    if i in melhores_sentencas:
      texto += str(i).replace(i, f"<mark>{i}</mark>")
    else:
      texto += i
  display(HTML(f""" {texto} """))

In [ ]:
visualiza_resumo('teste', sentencas_originais, melhores_sentencas)

HTML(value='<h1> Resumo de texto - teste</h1>')

HTML(value=' <mark>A educação é um dos pilares fundamentais para o desenvolvimento de uma sociedade.</mark><ma…

In [ ]:
!pip install goose3
from goose3 import Goose

In [ ]:
g = Goose()
url = 'https://www1.folha.uol.com.br/poder/2024/03/moraes-da-48-horas-para-bolsonaro-explicar-hospedagem-em-embaixada-diz-defesa.shtml'
artigo = g.extract(url)

In [ ]:
artigo = artigo.cleaned_text
artigo

In [ ]:
sentecas_originais_01, melhores_sentencas_01, nota_sentencas_01 = sumarizar(artigo, 5, 3, 3)

In [ ]:
sentecas_originais_01

In [ ]:
melhores_sentencas_01

In [ ]:
nota_sentencas_01

In [ ]:
visualiza_resumo(artigo.title, sentecas_originais_01, melhores_sentencas_01)